In [1]:
#Adding imports and needed functions
import pandas as pd, numpy as np, yfinance as yf, matplotlib.pyplot as plt
from scipy import stats
from pandas.tseries.offsets import BMonthEnd, BusinessDay
from datetime import date
from Initialize import Initialize
import tabulate

def vola_calc(ts):
    vola_window = 20
    return ts.pct_change().rolling(vola_window).std().dropna().iloc[-1]

In [2]:
#Setting the Universe to Commodity ETFs
universe = pd.read_excel("Comm_Universe.xlsx", engine='openpyxl')
tickers_list = universe["Symbol"].values.tolist()
tickers_list

['NIB',
 'JO',
 'CORN',
 'GRU',
 'WEAT',
 'COW',
 'CANE',
 'SOYB',
 'CUT',
 'WOOD',
 'CGW',
 'FIW',
 'PHO',
 'DBP',
 'GLTR',
 'XME',
 'GLD',
 'GDX',
 'SLV',
 'PPLT',
 'PLTM',
 'PALL',
 'REMX',
 'CHIM',
 'UYM',
 'URA']

In [3]:
#Initializing the universe & factor we are researching via Objects
comm_research = Initialize('Commodities', 'Momentum', tickers_list)

Initialize.displaySelection(comm_research)

total_hist = yf.download(tickers=comm_research.universe, period="1y",
                         interval="1d", group_by='ticker',
                         auto_adjust=True, prepost=True,
                         threads=True, proxy=None)

Name:  Commodities 
Alpha Factor:  Momentum 
Universe:  ['NIB', 'JO', 'CORN', 'GRU', 'WEAT', 'COW', 'CANE', 'SOYB', 'CUT', 'WOOD', 'CGW', 'FIW', 'PHO', 'DBP', 'GLTR', 'XME', 'GLD', 'GDX', 'SLV', 'PPLT', 'PLTM', 'PALL', 'REMX', 'CHIM', 'UYM', 'URA']
[*********************100%***********************]  26 of 26 completed


In [4]:
#Getting returns MTD & YTD
d=date.today()
idx = pd.IndexSlice
year_start = total_hist.loc['2021-12-31',idx[:,'Close']]

offset = BusinessDay(n=0)
curr_bd = (d - offset).strftime('%Y-%m-%d')
current = total_hist.loc[curr_bd,idx[:,'Close']]

ytd_df = pd.merge(year_start,current, right_index = True, left_index = True)
ytd_df['YTD Return %'] = (ytd_df[curr_bd]- ytd_df['2021-12-31']) / ytd_df['2021-12-31']

offset = BMonthEnd()

prev_mtd = offset.rollback(d).strftime('%Y-%m-%d')
prev_mtd_srs = total_hist.loc[prev_mtd,idx[:,'Close']]

mtd_df = pd.merge(prev_mtd_srs,current, right_index = True, left_index = True)
mtd_df['MTD Return %'] = (mtd_df[curr_bd]- mtd_df[prev_mtd]) / mtd_df[prev_mtd]

returns_df = pd.DataFrame(index=mtd_df.index)

returns_df['MTD Return %'],returns_df['YTD Return %'] = mtd_df['MTD Return %'],ytd_df['YTD Return %']

returns_df = returns_df.droplevel(level=1).sort_values(by = 'YTD Return %', ascending = False)

returns_df

,MTD Return %,YTD Return %
SOYB,-0.001081,0.218462
CORN,-0.015515,0.207525
XME,-0.075190,0.124863
GRU,-0.007505,0.118393
COW,0.004855,0.054811
CANE,0.004224,0.033696
WEAT,-0.052239,0.031123
PPLT,-0.036523,0.027030
PLTM,-0.038614,0.024910
SLV,0.045037,-0.007434


In [5]:
#Generating a table to display 1,3,6 Month Alpha Factor Research
dict_cols = {20: '1-Month',
             60: '3-Months',
             125: '6-Months',
             '20-corr': '1-Month Corr',
             '60-corr': '3-Months Corr',
             '125-corr': '6-Months Corr'}

time_list = [20, 60, 125]
r2_table = pd.DataFrame(columns=time_list, index=tickers_list).sort_index()

for time in time_list:
    ranking_t, hist_for_corr = Initialize.returnAlpha(comm_research, time, total_hist)

    r2_ranks = pd.DataFrame(ranking_t).sort_index()
    r2_table[time] = r2_ranks[0] / (125 / time)

    r2_table[str(time) + '-corr'] = np.nan

    for tick in tickers_list:
        r2_table.loc[tick, str(time) + '-corr'] = hist_for_corr.corr()[tick].drop(tick).mean()

r2_table.insert(3, 'Avg Alpha-Factor', r2_table[time_list].mean(axis=1))
r2_table = r2_table.rename(columns=dict_cols)
r2_table = r2_table.sort_values(by='Avg Alpha-Factor', ascending=False)

vola_table = hist_for_corr.apply(vola_calc)

r2_table['1-Month Vol'] = vola_table
r2_table.to_csv("r2_table_Comm.csv")

pd.set_option('display.max_columns', 1000000)
pd.set_option('display.max_rows', 1000000)

plt.rcParams.update({'figure.max_open_warning': 0})
r2_table

,1-Month,3-Months,6-Months,Avg Alpha-Factor,1-Month Corr,3-Months Corr,6-Months Corr,1-Month Vol
UYM,0.020557,259.468131,3.688865,87.725851,0.368107,0.534436,0.469397,0.029451
SLV,93.957322,53.965049,5.986299,51.302890,0.332946,0.512219,0.372916,0.021311
GDX,16.227832,89.371275,0.012406,35.203838,0.441126,0.536732,0.357514,0.024761
XME,0.102175,66.561061,11.420085,26.027774,0.349686,0.523365,0.428683,0.021994
PLTM,4.035774,48.749219,16.610986,23.131993,0.440326,0.534892,0.358616,0.017002
PPLT,3.575509,48.173768,16.517096,22.755458,0.437745,0.535273,0.357506,0.017269
CGW,0.379240,66.844230,0.092414,22.438628,0.415801,0.514223,0.413782,0.013243
CUT,3.391611,54.256660,-0.453697,19.064858,0.490741,0.524694,0.359671,0.010964
WOOD,0.124290,50.348829,-0.732142,16.580326,0.416261,0.525963,0.339923,0.011927
PHO,0.651366,44.944621,1.991561,15.862516,0.475599,0.521419,0.407465,0.013707
